##### Copyright 2018 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Custom layers

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/customization/custom_layers"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />Visualizza su TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/it/tutorials/customization/custom_layers.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Esegui in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/it/tutorials/customization/custom_layers.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />Visualizza il sorgente su GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/it/tutorials/customization/custom_layers.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Scarica il notebook</a>
  </td>
</table>

Raccomandiamo di usare `tf.keras` come API di alto livello per la costruzione di reti neurali. Detto ciò, molte delle API di TensorFLow sono utilizzabili tramite esecuzione eager.

In [ ]:
import tensorflow as tf

In [ ]:
print(tf.test.is_gpu_available())

## Layers: sets comuni di operazioni utili

Spesso scrivendo codice per modelli di machine learning vuoi operare a un livello di astrazione più alto delle singole operazioni e della manipolazione di singole variabili.

Molti modelli di machine learning sono esprimibili tramite composizione e impilamento di layer relativamente semplici e TensorFlow fornishe sia un set composto di molti layer comuni sia modalità semplici per te per scrivere layer specifici per la tua apllicazione da zero o come composizione di layer esistenti.

TensorFlow include l'API completa di [Keras](https://keras.io) nel package tf.keras e i layer di Keras sono molto utili quando si creano i propri modelli.


In [ ]:
# Nel package tf.keras.layers, i layers sono oggetti. Per costruire un layer,
# costruisci semplicemente l'oggetto. La maggior parte dei layer prendono come
# primo argomento il numero delle dimensioni / canali in output.
layer = tf.keras.layers.Dense(100)
# Il numero di dimensioni in input è spesso non necessario, dal momento che
# può essere inferita la prima volta che il layer è usato, ma può essere fornito
# se vuoi spcificarlo manualmente, il che è utile in alcuni modelli complessi.
layer = tf.keras.layers.Dense(10, input_shape=(None, 5))

La lista completa di layer preesistenti può essere vista nella [documentazione](https://www.tensorflow.org/api_docs/python/tf/keras/layers). Questa include Dense (un layer completamente connesso),
Conv2D, LSTM, BatchNormalization, Dropout e molti altri.

In [ ]:
# Per usare un layer, è sufficente chiamarlo.
layer(tf.zeros([10, 5]))

In [ ]:
# I layers hanno molti metodi utili. Per esempio, puoi ispezionare tutte le
# variabili in un layer usando `layer.variables` e variabili allenabili usando
# `layer.trainable_variables`. In questo caso un layer completamente connesso
# avrà variabili per pesi e bias.
layer.variables

In [ ]:
# Le variabili sono  anche accessibili tramite accessori.
layer.kernel, layer.bias

## Implementare layer custom

Il modo migliore per implementare il tuo proprio layer è estendere la classe tf.keras.Layer e implementare:

1. `__init__` , dove puoi fare tutte le inizializzazioni indipendenti dall'input
2. `build`, dove sai la forma dei tensori di input e puoi fare il resto dell'inizializzazione
3. `call`, dove puoi fare il calcolo forward

Nota che non devi aspettare fino a che `build` sia chiamata per creare le tue variabili, puoi anche crearle in `__init__`. Tuttavia, il vantaggio di creare in `build` sta nella possibilità di creare le variabili dopo, basandosi sulla forma dell'input su cui il layer opererà. Dall'altro lato, creare le variabili in `__init__` significherebbe che la forma richiesta per creare le variabili dovra' esspere esplicitamente specificata.

In [ ]:
class MyDenseLayer(tf.keras.layers.Layer):
  def __init__(self, num_outputs):
    super(MyDenseLayer, self).__init__()
    self.num_outputs = num_outputs

  def build(self, input_shape):
    self.kernel = self.add_weight("kernel",
                                  shape=[int(input_shape[-1]),
                                         self.num_outputs])

  def call(self, input):
    return tf.matmul(input, self.kernel)

layer = MyDenseLayer(10)

In [ ]:
_ = layer(tf.zeros([10, 5])) # Calling the layer `.builds` it.

In [ ]:
print([var.name for var in layer.trainable_variables])

Il codice in generale è più facile da leggere e mantenere se usa layer standard ogniqualvolta possibile, dal momento che i lettori saranno più familari col comportamento di layer standard. Se vuoi utilizzare un layer che non è present in `tf.keras.layers`, considera di creare una [issue github](http://github.com/tensorflow/tensorflow/issues/new) o, ancora meglio, di mandarci una pull request!

## Modelli: Comporre layers

Molte cose interessanti sullo stile dei layer nei modelli di machine learning sono implementate componendo layer esistenti. Per esempio, ogni blocco residuale in una resnet è la composizione di convoluzioni, normalizzazioni di batch e una scorciatoia. I layers possono essere innestati dentro altri layer.

Tipicamente erediti da `keras.Model` quando necessiti di metodi per i modelli tra i quali: `Model.fit`,`Model.evaluate` e `Model.save` (vedi [Layer Custom Keras e modelli](../../guide/keras/custom_layers_and_models.ipynb) per dettagli).

Un altra feature fornita da `keras.Model` (invece di `keras.layers.Layer`) è che in aggiunta al tenere traccia delle variabii, un `keras.Model` tiene traccia anche dei suoi layer interni, rendendoli più semplici per l'ispezione,

Per esempio ecco qui un blocco ResNet:

In [ ]:
class ResnetIdentityBlock(tf.keras.Model):
  def __init__(self, kernel_size, filters):
    super(ResnetIdentityBlock, self).__init__(name='')
    filters1, filters2, filters3 = filters

    self.conv2a = tf.keras.layers.Conv2D(filters1, (1, 1))
    self.bn2a = tf.keras.layers.BatchNormalization()

    self.conv2b = tf.keras.layers.Conv2D(filters2, kernel_size, padding='same')
    self.bn2b = tf.keras.layers.BatchNormalization()

    self.conv2c = tf.keras.layers.Conv2D(filters3, (1, 1))
    self.bn2c = tf.keras.layers.BatchNormalization()

  def call(self, input_tensor, training=False):
    x = self.conv2a(input_tensor)
    x = self.bn2a(x, training=training)
    x = tf.nn.relu(x)

    x = self.conv2b(x)
    x = self.bn2b(x, training=training)
    x = tf.nn.relu(x)

    x = self.conv2c(x)
    x = self.bn2c(x, training=training)

    x += input_tensor
    return tf.nn.relu(x)


block = ResnetIdentityBlock(1, [1, 2, 3])

In [ ]:
_ = block(tf.zeros([1, 2, 3, 3])) 

In [ ]:
block.layers

In [ ]:
len(block.variables)

In [ ]:
block.summary()

La maggior parte delle volte, tuttavia, i modelli che compongono molti layer sono semplicemente chiamati un layer alla volta. Questo può essere fatto con poco codice usando `tf.keras.Sequential`:

In [ ]:
my_seq = tf.keras.Sequential([tf.keras.layers.Conv2D(1, (1, 1),
                                                    input_shape=(
                                                        None, None, 3)),
                             tf.keras.layers.BatchNormalization(),
                             tf.keras.layers.Conv2D(2, 1,
                                                    padding='same'),
                             tf.keras.layers.BatchNormalization(),
                             tf.keras.layers.Conv2D(3, (1, 1)),
                             tf.keras.layers.BatchNormalization()])
my_seq(tf.zeros([1, 2, 3, 3]))

In [ ]:
my_seq.summary()

# Passi successivi

Ora puoi tornare indietro al notebook precedente e addattare l'esempio della regressione lineare per usare layer e modelli affiché sia meglio strutturato.